In [1348]:
# CODE HERE:
import matplotlib.pyplot as plt 
import matplotlib.image as img
import numpy as np
import cv2

In [1349]:
# reading in source image
# READING IMAGES:
in_img = cv2.imread('./ASSETS/fe.png', cv2.IMREAD_GRAYSCALE)
r_img = cv2.resize(in_img, (256, 256))
a_img = np.array(r_img)
n_img = ((a_img - np.min(a_img)) * (1/(np.max(a_img) - np.min(a_img)) * 255)).astype('uint8')
convolution_input = n_img.astype(float)

In [1350]:
# FUNCTIONS 01:
def convolution(image, filter):
    # Convert image to float array if it's not already
    image = [[float(val) for val in row] for row in image]
    # Convert filter to float array if it's not already
    filter = [[float(val) for val in row] for row in filter]
    
    # Determine the dimensions of the image and filter
    image_height = len(image)
    image_width = len(image[0])
    filter_height = len(filter)
    filter_width = len(filter[0])
    
    # Calculate padding
    pad_height = filter_height // 2
    pad_width = filter_width // 2
    
    # Pad the image with zeros on all sides
    padded_image = [[0 for _ in range(image_width + 2 * pad_width)] for _ in range(image_height + 2 * pad_height)]
    for i in range(image_height):
        for j in range(image_width):
            padded_image[i + pad_height][j + pad_width] = image[i][j]
    
    # Prepare the img_conv array
    img_conv = [[0 for _ in range(image_width)] for _ in range(image_height)]
    
    # Apply the filter
    for y in range(image_height):
        for x in range(image_width):
            # Extract the current region of interest
            region = [[padded_image[i][j] for j in range(x, x + filter_width)] for i in range(y, y + filter_height)]
            # Perform element-wise multiplication and sum the result
            img_conv[y][x] = sum(sum(region[i][j] * filter[i][j] for j in range(filter_width)) for i in range(filter_height))
    
    return img_conv

def manual_threshold(img_in, threshold):
    
    manual_thresh_img = []
    
    for row in img_in:
        thresholded_row = []
        for pixel in row:
            thresholded_row.append(255 if pixel > threshold else 0)
        manual_thresh_img.append(thresholded_row)

    threshold_img = manual_thresh_img

    return threshold_img

def gaussian_kernel_x(l, sig):
    
    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    gauss = np.exp(-0.5 * ax**2 / sig**2)
    kernel = np.outer(gauss, 1)

    return kernel / np.sum(kernel)

def gaussian_kernel_y(l, sig):
    
    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    gauss = np.exp(-0.5 * ax**2 / sig**2)
    kernel = np.outer(1, gauss)

    return kernel / np.sum(kernel)

In [1388]:
# A1:
# Derivative Kernels:
sigma = 1.0
filter_x = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
filter_y = [[1, 2, 1], [0, 0, 0], [-1, -2, -1]]
kernel_1dx = gaussian_kernel_x(6*int(sigma)+1, sigma)
kernel_1dy = gaussian_kernel_y(6*int(sigma)+1, sigma)

In [1389]:
gaussian_image_x = convolution(convolution_input, kernel_1dx)
gaussian_image = convolution(gaussian_image_x, kernel_1dy)
gradient_x = convolution(gaussian_image, filter_x)
gradient_y = convolution(gaussian_image, filter_y)
grad_mag = np.sqrt(np.array(gradient_x)**2 + np.array(gradient_y)**2)
cv2.imwrite('gradient_magnitude.png', grad_mag)


True

In [1390]:
def invert_gradient_intensities(grad_magnitude):
    
    cv2.normalize(grad_magnitude, grad_magnitude, 0, 1, cv2.NORM_MINMAX)
    
    inverted_grad_magnitude = 1 - grad_magnitude
    
    inverted_grad_magnitude = (inverted_grad_magnitude * 255).astype(np.uint8)
    
    return inverted_grad_magnitude

In [1391]:
inv_mag = invert_gradient_intensities(grad_mag)

cv2.imwrite('inv_gradient_magnitude.png', inv_mag)

True

In [1392]:
# FUNCTIONS 02:
def draw_contour(image, contour):
    result_img = np.zeros_like(image)
    res_img = cv2.polylines(result_img, [contour], isClosed=True, color=(255, 255, 255), thickness=2)
    return res_img

def overlay_images(image1, image2, alpha=.50):
    if not (0 <= alpha <= 1):
        raise ValueError("Alpha should be a float between 0 and 1.")
    
    # Calculate beta (the weight of the second image)
    beta = 1.0 - alpha
    
    blended_image = cv2.addWeighted(image1, alpha, image2, beta, 0)
    return blended_image


In [1393]:
points = []

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Append the new point (x, y)
        points.append((x, y))
        
        # Draw a small circle at the click point
        cv2.circle(img, (x, y), 3, (255, 255, 255), -1)
        
        # Draw the polygon if there are at least two points
        if len(points) >= 2:
            cv2.polylines(img, [np.array(points)], isClosed=False, color=(255, 255, 255), thickness=2)
        
        cv2.imshow('image', img)

# Load an image
img = grad_mag
if img is None:
    print("Error: Image not found.")
else:
    # Create a window
    cv2.namedWindow('image')
    cv2.setMouseCallback('image', click_event)

    # Display the image
    cv2.imshow('image', img)

    # Wait until any key is pressed
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [1394]:
contour = np.array(points ,dtype=np.int32)
contour_img = overlay_images(draw_contour(grad_mag, contour), convolution_input)


cv2.imwrite('init_contour.png', contour_img)

True

In [1395]:
contour_image = contour_img
init_contour = contour

In [1432]:
def snake_algorithm(gradient_image, initial_contour, iterations, alpha=0.1, beta=0.61, gamma=.20, energy_threshold=4.0):
    inverted_gradient = 255 - gradient_image

    snake = initial_contour.astype(np.float32)

    gray_image = cv2.cvtColor(inverted_gradient, cv2.COLOR_BGR2GRAY) if len(inverted_gradient.shape) == 3 else inverted_gradient

    gray_image_8bit = cv2.convertScaleAbs(gray_image)

    external_force = cv2.distanceTransform(gray_image_8bit, cv2.DIST_L2, 5)
    internal_energy = 0
    external_energy = 0

    # Iteratively update the snake
    for i in range(iterations):

        
        # Internal force (continuity and curvature)
        for i in range(snake.shape[0]):
            prev_point = snake[i - 1 if i - 1 >= 0 else -1]
            next_point = snake[(i + 1) % snake.shape[0]]

            continuity_force = alpha * (prev_point - snake[i])
            curvature_force = beta * (prev_point - 2 * snake[i] + next_point)

            snake[i] += gamma * (continuity_force + curvature_force)

            

        # External force (image gradient)
        for i in range(snake.shape[0]):
            point = np.round(snake[i]).astype(int)
           
            point[0] = np.clip(point[0], 0, external_force.shape[1] - 1)
            point[1] = np.clip(point[1], 0, external_force.shape[0] - 1)

            # Apply the external force
            gradient = np.array(np.gradient(external_force))
            external_force_at_point = gradient[:, point[1], point[0]]
            snake[i] += gamma * external_force_at_point

        internal_energy += continuity_force**2 + curvature_force**2    
        external_energy += np.linalg.norm(external_force_at_point)**2
        
          
    snake = snake.astype(np.int32)
    

    return snake



In [1433]:
snake_contour_01 = snake_algorithm(inv_mag, init_contour, 5)
snake_contour_02 = snake_algorithm(inv_mag, init_contour, 10)
snake_contour_03 = snake_algorithm(inv_mag, init_contour, 15)
snake_contour_04 = snake_algorithm(inv_mag, init_contour, 20)
snake_contour_05 = snake_algorithm(inv_mag, init_contour, 25)
snake_contour_06 = snake_algorithm(inv_mag, init_contour, 30)



In [1434]:
snake_contour_img1 = overlay_images(draw_contour(grad_mag, snake_contour_01), convolution_input)
snake_contour_img2 = overlay_images(draw_contour(grad_mag, snake_contour_02), convolution_input)
snake_contour_img3 = overlay_images(draw_contour(grad_mag, snake_contour_03), convolution_input)
snake_contour_img4 = overlay_images(draw_contour(grad_mag, snake_contour_04), convolution_input)
snake_contour_img5 = overlay_images(draw_contour(grad_mag, snake_contour_05), convolution_input)
snake_contour_img6 = overlay_images(draw_contour(grad_mag, snake_contour_06), convolution_input)

cv2.imwrite('step1.png', snake_contour_img1)
cv2.imwrite('step2.png', snake_contour_img2)
cv2.imwrite('step3.png', snake_contour_img3)
cv2.imwrite('step4.png', snake_contour_img4)
cv2.imwrite('step5.png', snake_contour_img5)
cv2.imwrite('step6.png', snake_contour_img6)

True